# Bot de música
En esta clase veremos cómo hacer un bot básico de música.

<div>
<img src="images/music.jpg" width="600" align="center"/>
</div>

## 1.- Entrar y salir al canal de voz
Primero veamos cómo hacer que el bot ingrese y salga al canal de voz.


Para ingresar a un canal, basta hacer

```
channel = ctx.author.voice.channel
await channel.connect()```

Donde `ctx` es el contexto del comando.

Para salir

`await ctx.voice_client.disconnect()`

Donde `ctx` es el contexto del comando.

Antes de ocupar los comandos presentados se debe instalar la libería `PyNaCl`. 

`pip3 install PyNaCl`

In [ ]:
@bot.command()
async def join(ctx):
    channel = ctx.author.voice.channel
    await channel.connect()

@bot.command()
async def leave(ctx):
    await ctx.voice_client.disconnect()

## 2.- Reproducir mp3
Ahora veremos cómo hacer que el bot reproduzca un archivo mp3.

Para reproducir un audio, podemos ocupar el siguiente comando

`
source = discord.FFmpegPCMAudio('audio.mp3')
ctx.voice_client.play(source)`

Donde `ctx` es el contexto del comando. `audio.mp3` se reemplaza por el nombre del archivo de audio que queremos reproducir.

In [ ]:
@bot.command()
async def play(ctx):
    # nos aseguramos de que no se este re
    source = discord.FFmpegPCMAudio('audio.mp3')
    ctx.voice_client.play(source)

## 3.- Descargar audio desde Youtube
La implementación que vamos a usar es descargar el archivo de audio de del video de YouTube que coloquemos y desde ese archivo nuestro bot va a reproducir la canción.

Se va a ocupar la libería `youtube_dl` que nos permite descargar en formato de audio videos de YouTube.

Copía y pega el siguiente bloque sacado de este [tutorial](https://medium.com/pythonland/build-a-discord-bot-in-python-that-plays-music-and-send-gifs-856385e605a1).

In [ ]:
import youtube_dl

youtube_dl.utils.bug_reports_message = lambda: ''

ytdl_format_options = {
    'format': 'bestaudio/best',
    'restrictfilenames': True,
    'noplaylist': False,
    'nocheckcertificate': True,
    'ignoreerrors': False,
    'logtostderr': False,
    'quiet': True,
    'no_warnings': True,
    'default_search': 'auto',
    'source_address': '0.0.0.0' # bind to ipv4 since ipv6 addresses cause issues sometimes
}

ffmpeg_options = {
    'options': '-vn'
}

ytdl = youtube_dl.YoutubeDL(ytdl_format_options)

class YTDLSource(discord.PCMVolumeTransformer):
    def __init__(self, source, *, data, volume=0.5):
        super().__init__(source, volume)
        self.data = data
        self.title = data.get('title')
        self.url = ""

    @classmethod
    async def from_url(cls, url, *, loop=None, stream=False):
        loop = loop or asyncio.get_event_loop()
        data = await loop.run_in_executor(None, lambda: ytdl.extract_info(url, download=not stream))
        if 'entries' in data:
            # take first item from a playlist
            data = data['entries'][0]
        filename = data['title'] if stream else ytdl.prepare_filename(data)
        return filename

`YTDLSource` es una clase que descarga un archivo de audio desde el link de un video de YouTube.

Entonces para reproducir un video:

`filename = await YTDLSource.from_url(url, loop=bot.loop)
 voice_channel.play(discord.FFmpegPCMAudio(source=filename))`

In [ ]:
@bot.command()
async def play_url(ctx,url):
    filename = await YTDLSource.from_url(url, loop=bot.loop)
    ctx.voice_client.play(discord.FFmpegPCMAudio(source=filename))

## 4.- Reproducción por stream
Para no necesitar descargar el archivo se hace la siguiente modificación a la clase `YTDLSource`

en el metodo `from_url` cambiamos el argumento de `stream=False` a `stream=True` y cambiamos el return a:

`return cls(discord.FFmpegPCMAudio(filename, **ffmpeg_options), data=data)`

Y cambiamos el comando `play_url` a:

In [ ]:
@bot.command()
async def play_url(ctx, url):
    player = await YTDLSource.from_url(url, loop=bot.loop)
    ctx.voice_client.play(player)

## 5.- Ejercicio: pausa, continuar y parar
Busquen cómo se implementa esto ayudandose de la documentación respectiva de [VoiceClient](https://discordpy.readthedocs.io/en/stable/api.html?highlight=play#voiceclient).